# 인구 조사 데이터로 인종 예측하기

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [124]:
df = pd.read_csv('./datasets/adult.data', header=None)
df.shape

(32561, 15)

In [125]:
df.isna().sum().sum()

0

In [126]:
race = df[8]
set(race)

{' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'}

In [127]:
x = df.drop(columns=[2, 8]) # 의미없는 2번 컬럼과 y 컬럼 제외
y = race.map({' Amer-Indian-Eskimo': 0, ' Asian-Pac-Islander': 1,
         ' Black':2, ' Other':3, ' White':4})
x.shape, y.shape

((32561, 13), (32561,))

1. 누락 데이터 처리
1. X, Y 분리, 카테고리를 원핫 인코딩
1. 배열로 변환, float32 타입 변환
1. 사이킷런 LogisticRegression 으로 학습 및 성능 체크
1. 텐서플로 직접 구현

In [128]:
x_dum = pd.get_dummies(x)
x_dum = np.array(x_dum).astype(np.float32)
y = np.array(y)
x_dum.shape, type(x_dum), type(y) # 텐서플로는 numpy 만 읽을 수 있다

((32561, 104), numpy.ndarray, numpy.ndarray)

In [129]:
from sklearn.linear_model import LogisticRegression
#model = LogisticRegression(solver='lbfgs', max_iter=200,multi_class='auto').fit(x_dum, y)
model = LogisticRegression().fit(x_dum, y)
print('accuracy', model.score(x_dum, y))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 0.8734989711618194


In [ ]:
from sklearn.preprocessing import OneHotEncoder
y_2d = y.reshape([-1,1]) 
y_hot = OneHotEncoder().fit_transform(y_2d) # # tf.one_hot() 과 같다
y_hot = y_hot.toarray() # sparse matrix > dense matrix
y_hot.shape

In [ ]:
w = tf.Variable(tf.random_normal(shape=[104, 5])) #[104개 특징, 5개의 클래스] 
b = tf.Variable(tf.random_normal(shape=[5]))#5개의 클래스 
xn = x_dum / tf.reduce_max(x_dum)
z = tf.matmul(xn, w) + b 
h = tf.nn.softmax(z)
h = tf.clip_by_value(h, 1e-8, 1 - 1e-8) # log(0) 이 되지 않도록 안전장치
cost = - tf.reduce_mean(y_hot * tf.log(h) + (1-y_hot) * tf.log(1-h))
regularization = 1e-3 * tf.reduce_mean(w**2)
predict_class = tf.math.argmax(h, axis=1, output_type=tf.int32) #[0, 2]
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict_class, y), tf.float32)) 
train = tf.train.GradientDescentOptimizer(0.5).minimize(cost+regularization)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(1000):
    _, c, a = sess.run([train, cost, accuracy])
    if i%100 == 0: print(i, c, a)
print('w : 상관관계 ', sess.run(w))

In [79]:
weight = sess.run(w)
weight.shape

(104, 5)

In [80]:
col_name = pd.get_dummies(x).columns.values
len(col_name), col_name.shape

(104, (104,))

In [81]:
d = pd.DataFrame(weight)

In [82]:
d.columns = set(race)
d.set_index(col_name)

,Asian-Pac-Islander,Other,White,Amer-Indian-Eskimo,Black
0,-0.626123,-0.839252,-0.568518,-0.965661,-0.562979
4,-0.392450,-0.653408,-0.621373,0.593650,-1.165223
10,-1.338175,0.395253,0.192901,-1.106798,0.668396
11,0.188518,-0.342175,0.280743,-0.480473,1.498428
12,-2.101917,-0.203840,0.772987,1.056976,0.308852
1_ ?,-0.323664,0.398586,-1.663887,-1.021099,1.183541
1_ Federal-gov,1.145826,-1.560326,-0.074274,-0.649132,0.969502
1_ Local-gov,2.049340,1.079450,0.714968,0.960439,-0.302685
1_ Never-worked,0.507475,-0.150548,-1.848548,0.333915,-1.319711
1_ Private,-1.355100,2.136464,-0.644070,1.244481,0.272604


## 자동차 검사 결과 예측하기

In [86]:
df = pd.read_csv('./datasets/car.data', header=None)
df.shape

(1728, 7)

In [91]:
df.columns = ['buying price', 'maintenance','doors','persons','lug_boot','safe','y']

In [93]:
df.head()

,buying price,maintenance,doors,persons,lug_boot,safe,y
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [ ]:
df.isna().sum().sum()

In [ ]:
df_dum = pd.get_dummies(df)
df_dum.columns

In [104]:
x = np.array(df_dum.iloc[:, :-4])
y = np.array(df_dum.iloc[:, -4:])
y_argmax = np.argmax(y, axis=-1)
x.shape, y.shape

((1728, 21), (1728, 4))

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(x, y_argmax)
print('accuracy', model.score(x, y_argmax))

In [113]:
w = tf.Variable(tf.random_normal(shape=[21, 4])) #[21개 특징, 4개의 클래스] 
b = tf.Variable(tf.random_normal(shape=[4]))#4개의 클래스 
xn = x / tf.reduce_max(x)
z = tf.matmul(xn, w) + b 
h = tf.nn.softmax(z)
h = tf.clip_by_value(h, 1e-8, 1 - 1e-8) # log(0) 이 되지 않도록 안전장치
cost = - tf.reduce_mean(y * tf.log(h) + (1-y) * tf.log(1-h))
regularization = 1e-3 * tf.reduce_mean(w**2)
predict_class = tf.math.argmax(h, axis=1, output_type=tf.int32) #[0, 2]
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict_class, y_argmax), tf.float32)) 
train = tf.train.GradientDescentOptimizer(0.5).minimize(cost+regularization)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [114]:
for i in range(1000):
    _, c, a = sess.run([train, cost, accuracy])
    if i%100 == 0: print(i, c, a)

0 0.9495914 0.28819445
100 0.21634613 0.8142361
200 0.18358842 0.8553241
300 0.16657852 0.8709491
400 0.15436229 0.8767361
500 0.14531383 0.880787
600 0.13847715 0.8865741
700 0.13311775 0.8912037
800 0.1287559 0.8952546
900 nan 0.22222222
